<a href="https://colab.research.google.com/github/chadloh/GoogleDecimeterChallenge/blob/main/LayalHammad/mapanimated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#@title Packages { display-mode: "form" }
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import glob as gl 
import scipy.optimize
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance
from scipy import signal
import seaborn as sns

from tensorflow import keras
from keras import layers, models

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.recurrent import LSTM,RNN
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 


from matplotlib import animation, patches
# ! pip install lib
# from lib import error_ellipse
!pip install pymap3d
import pymap3d as pm
import pymap3d.vincenty as pmv

# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pymap3d-2.9.0-py3-none-any.whl


In [6]:
#@title Distance and Score Functions { display-mode: "form" }
# Compute distance by Vincenty's formulae
def vincenty_distance(llh1, llh2):
    """
    Args:
        llh1 : [latitude,longitude] (deg)
        llh2 : [latitude,longitude] (deg)
    Returns:
        d : distance between llh1 and llh2 (m)
    """
    d, az = np.array(pmv.vdist(llh1[:, 0], llh1[:, 1], llh2[:, 0], llh2[:, 1]))

    return d


# Compute score
def calc_score(llh, llh_gt):
    """
    Args:
        llh : [latitude,longitude] (deg)
        llh_gt : [latitude,longitude] (deg)
    Returns:
        score : (m)
    """
    d = vincenty_distance(llh, llh_gt)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])

    return score

In [7]:
import matplotlib.pyplot as plt
import time
from IPython import display
from xml.dom import minidom
import math  

In [7]:
# !pip uninstall kaleido

In [8]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:46
🔁 Restarting kernel...


In [1]:
!conda install -c conda-forge python-kaleido

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - python-kaleido


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py37h89c1867_0         155 KB  conda-forge
    conda-4.13.0               |   py37h89c1867_1         989 KB  conda-forge
    expat-2.2.10               |       h9c3ff4c_0         164 KB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94 KB  conda-forge
    font-ttf-source-code-pro-2.038|       h77eed37_0         684 KB  conda-forge
    font-ttf-ubuntu-0.83       |       hab24e00_0         1.9

In [8]:
g = pd.read_csv('/content/drive/MyDrive/2020-12-10-US-SJC-1GooglePixel4.csv', index_col= 0)

In [9]:
g.columns

Index(['MessageType', 'Provider', 'lat', 'lon', 'AltitudeMeters', 'speed',
       'AccuracyMeters', 'BearingDegrees', 'UnixTimeMillis', 'time', 'latgnss',
       'longnss', 'Error', 'dist', 'spd'],
      dtype='object')

In [13]:
rating = []
for row in g['Error']:
  if row > 5.0 :    rating.append(5)
  else:           
    rating.append(row)                                             


In [14]:
g['E'] = rating

In [15]:
fig = px.scatter_mapbox(g, lat = g['latgnss'] , lon=g['longnss'], color=g["E"], height=400, width=1200)
fig.layout.coloraxis.colorbar.title = 'Error'
fig.update_layout(
    title='Visualizing GNSS Baseline Error',
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken='pk.eyJ1IjoiY2hhZGxvaCIsImEiOiJjbDF5aTR2aXYwN2U0M2p0Y3kzdjQ5cmhrIn0.ACJNflNkzHgMJ9Ou1jFNaA',
        bearing=0,
        center=dict(
            lat=37.3168,
            lon=-121.962
        ),
        pitch=0,
        zoom=14.25,
        style='light'
    ),
)


fig.add_trace(
      go.Scattermapbox(
          name='ground truth',
          lon = g['lon'],
          lat = g['lat'],
          mode = 'lines',
          showlegend = False,
          line = dict(width = 1,color = 'grey')))


In [16]:
import kaleido
u = 0

for i in range(g.shape[0]):
  fig = px.scatter_mapbox(g, lat = g['lat'][0:u] , lon=g['lon'][0:u], color=g["E"][0:u], height=400, width=1200, range_color=(0,5))
  fig.layout.coloraxis.colorbar.title = 'Error'
  fig.update_layout(
      title='Visualizing GNSS Baseline Error',
      hovermode='closest',
      showlegend=True,
      mapbox=dict(
          accesstoken='pk.eyJ1IjoiY2hhZGxvaCIsImEiOiJjbDF5aTR2aXYwN2U0M2p0Y3kzdjQ5cmhrIn0.ACJNflNkzHgMJ9Ou1jFNaA',
          bearing=0,
          center=dict(
              lat=37.3168,
              lon=-121.962
          ),
          pitch=0,
          zoom=14.25,
          style='light'
      ),
  )
  fig.add_trace(
        go.Scattermapbox(
            name='ground truth',
            lon = g['lon'],
            lat = g['lat'],
            mode = 'lines',
            showlegend = False,
            line = dict(width = 1,color = 'grey')))
  u+=20
  fig.write_image(f"/content/drive/MyDrive/images/fig{i}.png")  



KeyboardInterrupt: ignored

32512